In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time

from tqdm import tqdm
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, cross_validate
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
PATH = "/kaggle/input/datasetpurplehack/"

In [3]:
drop_col = np.load(PATH + 'drop_cols.npy').tolist()
drop_col1 = np.load(PATH + 'drop_cols1.npy').tolist()

drop_col2 = ['feature3', 'feature22', 'feature53', 'feature73', 'feature110', 'feature140', 'feature150', 'feature164', 'feature166',
 'feature177', 'feature218', 'feature307', 'feature344', 'feature345', 'feature347', 'feature348', 'feature358',
 'feature377', 'feature379', 'feature436', 'feature445', 'feature453', 'feature515', 'feature552', 'feature572', 'feature26',
 'feature614', 'feature649', 'feature820', 'feature879', 'feature901', 'feature995', 'feature1046', 'feature1074']

df = pd.read_parquet(PATH + "train_ai_comp_final_dp.parquet").drop(columns=['sample_ml_new',]+drop_col+drop_col1+drop_col2)
test = pd.read_parquet("/kaggle/input/testdatapurplehack/test_sber.parquet").drop(columns=['sample_ml_new',]+drop_col+drop_col1+drop_col2)

In [4]:
for d in [df, test]:
    d['feature954_feature713'] = d['feature954']*d['feature713']
    d['feature270_feature864'] = d['feature270']*d['feature864']
    d['feature349_feature270'] = d['feature349']*d['feature270']

    d['feature954_feature4'] = d['feature954']*d['feature4']
    d['feature954_feature714'] = d['feature954']*d['feature714']
    d['feature916_feature954'] = d['feature916']*d['feature954']
    d['feature713_feature954'] = d['feature713']*d['feature954']

In [5]:
X, y = df.drop(columns=['target',]), df['target']
X_test = test[X.columns]

In [6]:
start_time = time.time()

param = {'lambda': 0.0010369926252582237,
 'alpha': 1.0131345049272614e-08,
 'subsample': 0.9136533888474965,
 'colsample_bytree': 0.6212890844778247,
 'learning_rate': 0.0528863041801649,
 'n_estimators': 250,
 'max_depth': 8,
 'min_child_weight': 18,
 'gamma': 1.605198662413527e-08}

model1 = XGBClassifier(**param, seed=42, tree_method= 'gpu_hist',)
data_score = cross_validate(model1, X, y, cv=15, scoring='roc_auc', return_estimator=True)
print(np.mean(data_score['test_score']))

# xgboost = 0.7617924182982958, lb = 0.7740275339694382
print("--- %s seconds ---" % (time.time() - start_time))

0.7598405440851689
--- 60.35625767707825 seconds ---


In [8]:
predict_list = []

for e in data_score['estimator']:
    predict_list.append(e.predict_proba(X_test)[:, 1])

# ensemble = 0.77144

pred = np.mean(predict_list, axis=0)
pred_binary = (pred >= 0.1).astype(int)

submission = pd.read_csv("/kaggle/input/testdatapurplehack/sample_submission.csv")
submission["target_prob"] = pred
submission["target_bin"] = pred_binary
submission.to_csv("submission.csv", index=False) 